# Assignment 2: Card Draw

![Assignment 2 Image of Five Playing Cards](img/Card_Draw.png)

Author: Irene Kilgannon

Student ID: G00220627

## Task

Write a program that 'deals' i.e. prints out 5 cards from this API, https://deckofcardsapi.com/.

First shuffle the deck and get the deck_id. With the deck_id, get the cards. Print the value and suit of each card.

Extra marks: If the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.

In [238]:
# Import modules
import requests
import json
import pandas as pd
from emoji import emojize

## Shuffle The 'Cards'

In [239]:
shuffle_cards_url = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"

# Get data from the API
response = requests.get(shuffle_cards_url)

# Convert JSON response to Python dictionary
response_dict = response.json()

print('Deck ID:', response_dict['deck_id'])

Deck ID: fl9e1z9tybuw


In [240]:
# Create a variable of the deck_id
deck_id =  response_dict['deck_id']

## Deal Five Cards

According to the API, this URL, https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=5  will deal five cards using the deck_id with the parameter of count equal to five.

In [241]:
deal_cards_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=5"

# Get data from the deal_cards_URL
response2 = requests.get(deal_cards_url)

# Convert JSON response to Python dictionary
deal_cards = response2.json()

# Print the value and suit of each card
for card in deal_cards['cards']:
    print(f"{card['value']} of {card['suit']}")

10 of CLUBS
10 of DIAMONDS
ACE of SPADES
10 of SPADES
8 of DIAMONDS


In [242]:
# Write data to a JSON file
with open ("data/deal_cards.json", "w") as fp:
    json.dump(deal_cards, fp)

## Extra Marks

If the user has drawn a pair, triple, straight, or all cards of the same suit and congratulate the user.

In [243]:
# Convert to a DataFrame
df = pd.DataFrame(deal_cards['cards'], columns= ['value', 'suit'])
df

,value,suit
0,10,CLUBS
1,10,DIAMONDS
2,ACE,SPADES
3,10,SPADES
4,8,DIAMONDS


### Checking For a Pair Or Triple.

In [244]:
# Count the number of duplicate rows in the value column.
no_of_duplicated_values = df['value'].duplicated().sum()
print(f'No. of duplicated card values: {no_of_duplicated_values}')

No. of duplicated card values: 2


In [245]:
# Create a dataframe of the duplicated rows.
df2 = df[df['value'].duplicated(keep = False)]
df2

,value,suit
0,10,CLUBS
1,10,DIAMONDS
3,10,SPADES


By creating a separate dataframe of the duplicate rows, I can distinguish between a pair, a pair of pairs or a triple by the length of the dateframe and also return the value of the cards. 

In [253]:
# Check for a pair.
if no_of_duplicated_values == 1:
    print(f"{emojize(":partying_face:")} Congratulations!!! You have a pair of {df2.iloc[0, 0]}'s.")

# Check for 2 pairs or a triple.
elif no_of_duplicated_values == 2:
    # Check for a triple
    if len(df2) == 3:
        print(f"{emojize(":party_popper:")} Congratulations!!! You have a triple of {df2.iloc[0, 0]}'s.")
    # Check for 2 pairs
    else:
        print(f"{emojize(":party_popper:")} WOW! Congratulations!! You have a two pairs of {df2.iloc[0, 0]} and {df2.iloc[2, 0]}.")
else:
    print(f'{emojize(":disappointed_face:")} Unlucky, no pairs or triples. Better luck next time.')

🎉 Congratulations!!! You have a triple of 10's.


### Checking for Matching Suit.

In [247]:
# Check if all cards have the same suit.
if df['suit'].duplicated().sum() == 4:
    print(f'{emojize(":fireworks:")} All cards have the same suit. Congratulations! {emojize(":fireworks:")}')
else:
    print("The cards are not all of the same suit.")

The cards are not all of the same suit.


### Checking for a Straight.

In [248]:
# For straight, need an ordered list of card values. 
# Ace has a value of 1 and 11. 
card_value_order = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'JACK', 'QUEEN', 'KING', 'ACE', '2', '3', '4']

In [249]:
# Create a list of the card values from the dataframe.
values = df.iloc[:5, 0].tolist()
values

['10', '10', 'ACE', '10', '8']

In [250]:
# Sort the values list by the card_value_order list
# https://stackoverflow.com/questions/34308635/sort-elements-with-specific-order-in-python (Clickable link in References)
values = (sorted(values, key = card_value_order.index))
values

['8', '10', '10', '10', 'ACE']

In [251]:
# Check for the presence of the values list in card_values.
# https://stackoverflow.com/questions/3313590/check-for-presence-of-a-sliced-list-in-python (Clickable link in References)

n = len(values)

if any((values == card_value_order[i:i+n]) for i in range(len(card_value_order) - n + 1)):
    print(f"{emojize(":partying_face:")} You have a straight.")
else:
    print(f"{emojize(":disappointed_face:")} No straight found.")

😞 No straight found.


## References

[Geeksforgeeks.org, response.json() - Python requests](https://www.geeksforgeeks.org/response-json-python-requests/)

[W3 schools.com, Python Requests Module](https://www.w3schools.com/python/module_requests.asp)

[Realpython.com, Python's Requests Library (Guide)](https://realpython.com/python-requests/)

[Stackoverflow.com, How to access a dictionary key value present inside a list](https://stackoverflow.com/questions/6521892/how-to-access-a-dictionary-key-value-present-inside-a-list)

[Geeksforgeeks.org, Accessing nested dictionaries in a list](https://www.geeksforgeeks.org/accessing-value-inside-python-nested-dictionaries/) 

[Datacamp.com, importing json data and working with APIs](https://campus.datacamp.com/courses/streamlined-data-ingestion-with-pandas/importing-json-data-and-working-with-apis?ex=4)

[Get a list from a dataframe](https://saturncloud.io/blog/how-to-get-a-list-from-a-pandas-dataframe-column/#:~:text=Best%20Practices-,Use%20the%20tolist()%20Method,of%20data%20into%20a%20list)

[Stackoverflow, Sort elements with specific order in Python](https://stackoverflow.com/questions/34308635/sort-elements-with-specific-order-in-python)

[Stackoverflow, Check for presence of a sliced list in Python](https://stackoverflow.com/questions/3313590/check-for-presence-of-a-sliced-list-in-python)